In [1]:
import sys; sys.path.append("../../wwa"); from wwa import *

mapproj = cartopy.crs.PlateCarree()

fpath = "/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/FS_heat/"

Xn,Xx,Yn,Yx = [0,35,55,75]
xn,xx,yn,yx = [4,30,60,72]

# ERA5

In [ ]:
era5_fpath = fpath+"era5_tmax_daily_eu_extended_0-35E_55-75N_-999-lastyear.nc"
! wget https://climexp.knmi.nl/data/era5_tmax_daily_eu_extended_0-35E_55-75N_-999-lastyear_su.nc -O $era5_fpath

era5_fpath = fpath+"era5_tmin_daily_eu_extended_0-35E_55-75N_-999-lastyear.nc"
! wget https://climexp.knmi.nl/data/era5_tmin_daily_eu_extended_0-35E_55-75N_-999-lastyear_su.nc -O $era5_fpath

In [37]:
da = xr.open_dataset(fpath+"era5_tmax_daily_eu_extended_0-35E_55-75N_-999-lastyear.nc").tmax.where(lsm >= 0.5)
rm = regionmask.defined_regions.natural_earth_v5_0_0.land_50.mask(da.lon, da.lat)
da.where(rm == 0).to_netcdf(fpath+"era5_tmax_daily_land_0-35E_55-75N.nc")

In [38]:
da = xr.open_dataset(fpath+"era5_tmin_daily_eu_extended_0-35E_55-75N_-999-lastyear.nc").tmin.where(lsm >= 0.5)
rm = regionmask.defined_regions.natural_earth_v5_0_0.land_50.mask(da.lon, da.lat)
da.where(rm == 0).to_netcdf(fpath+"era5_tmin_daily_land_0-35E_55-75N.nc")

# E-Obs

## Retrieve latest (both resolutions for now, in case of gaps)

In [28]:
eobs_path = fpath+"tx_ens_mean_0.1deg_reg_v31.0e.nc"
! wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/Grid_0.1deg_reg_ensemble/tx_ens_mean_0.1deg_reg_v31.0e.nc -O $eobs_path

In [25]:
eobs_path = fpath+"tx_ens_mean_0.25deg_reg_v31.0e.nc"
! wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/Grid_0.25deg_reg_ensemble/tx_ens_mean_0.25deg_reg_v31.0e.nc -O $eobs_path

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
eobs_path = fpath+"tx_0.25deg_day_2025_grid_ensmean.nc"
! wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/months/ens/tx_0.25deg_day_2025_grid_ensmean.nc -O $eobs_path

In [ ]:
eobs_path = fpath+"tn_ens_mean_0.25deg_reg_v31.0e.nc"
! wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/Grid_0.25deg_reg_ensemble/tn_ens_mean_0.25deg_reg_v31.0e.nc -O $eobs_path

In [ ]:
eobs_path = fpath+"tn_ens_mean_0.1deg_reg_v31.0e.nc"
! wget https://knmi-ecad-assets-prd.s3.amazonaws.com/ensembles/data/Grid_0.1deg_reg_ensemble/tn_ens_mean_0.1deg_reg_v31.0e.nc -O $eobs_path

## Extract wider region

In [2]:
varnm = "tn"
res = "0.25"

In [3]:
ds = xr.open_mfdataset([fnm for fnm in glob.glob(fpath+varnm+"*"+res+"*.nc") if not "lastyear" in fnm])
ds = ds[list(ds.data_vars)[0]].sel(longitude = slice(Xn,Xx), latitude = slice(Yn,Yx))
ds = ds.dropna("time", "all")

In [4]:
ds.to_netcdf(fpath+"eobs_"+varnm+"_"+res+"deg.nc")

# CPC

In [ ]:
! cd $fpath; wget https://climexp.knmi.nl/data/tmin_cpc_daily_0-35E_55-75N_-999-lastyear_su.nc
! cd $fpath; wget https://climexp.knmi.nl/data/tmax_cpc_daily_0-35E_55-75N_-999-lastyear_su.nc

# MSWX

In [2]:
outpath = "/rds/general/user/cb2714/home/99_ephemera/mswx_fs/"

In [ ]:
for y in range(2025,1978,-1):

    print(y)
    new_fnm = outpath+"tmax_mswx_"+str(y)+".nc"
    if os.path.exists(new_fnm): continue
        
    ds = xr.open_mfdataset("/rds/general/user/cb2714/home/00_WWA_project_folder/live/data/mswx/Tmax/"+str(y)+"/*.nc")
    ds = ds.sel(lon = slice(Xn,Xx), lat = slice(Yx,Yn)).rename(air_temperature = "tmax")
    ds.to_netcdf(new_fnm)

In [23]:
# compile & save
wrap_lon(xr.open_mfdataset(outpath+"*.nc")).to_netcdf("data/tmax_mswx.nc")